# CS918-Exercise 1
## Text preprocessing, N-grams and language models
### Nan.Liu.1@warwick.ac.uk

### Define functions
Function **remove_nonalpha()** is for removing all non-alphanumeric characters except spaces.  
Function **remove_character()** is for removing words with only 1 character.  
Function **remove_digit()** is for removing numbers that are fully made of digits.  
Function **remove_url()** is for removing URLs.  
Function **count_opinion_word()** is for counting the number of positive and negative words.  
Function **compute_freq_dist()** is for counting the frequency of every n-gram in given n-gram list.  
Function **sort_freqdist()** is for sorting the frequency of every ngram and get the top n ngrams based on the number of occurrences.

In [3]:
import json
import re
import time
import nltk
import math


# Function - Part A
def remove_nonalpha(string):
    """Function remove_nonalpha() is for removing all non-alphanumeric characters except spaces."""
    pattern = re.compile('[^a-z0-9 ]+')
    newstring = pattern.sub(' ', string)
    return newstring


def remove_character(string):
    """Function remove_character() is for removing words with only 1 character."""
    pattern = re.compile('\\b[a-z]\\b')
    newstring = pattern.sub(' ', string)
    newstring = re.sub('[ ]{2,}', ' ', newstring)
    return newstring


def remove_digit(string):
    """Function remove_digit() is for removing numbers that are fully made of digits."""
    pattern = re.compile('\\b[0-9]+\\b')
    newstring = pattern.sub(' ', string)
    newstring = re.sub('[ ]{2,}', ' ', newstring)
    return newstring


def remove_url(string):
    """Function remove_url() is for removing URLs."""
    pattern = re.compile('((https?):\/\/)?([a-z0-9]+([\.\/\?\=\&\*\%\-]+[a-z0-9]+)+)')
    newstring = pattern.sub(' ', string)
    return newstring


# Function - Part B
def count_opinion_word(word_list, opinion_dict):
    """
        Function count_opinion_word() is for counting the number of positive and negative words.
        input:  word_list is a list needed to count positive and negative words
                opinion_dict is a dictionary containing positive and negative words
        output: the numbers of negative and positive words in the given word_list
    """
    num_neg = 0
    num_pos = 0
    for word in word_list:
        if word in opinion_dict.keys():
            if opinion_dict[word] == 1:
                num_pos += 1
            else:
                num_neg += 1
    return num_neg, num_pos


def compute_freq_dist(ngrams):
    """
        Function compute_freq_dist() is for counting the frequency of every n-gram in given n-gram list.
        input:  ngrams is the return value of nltk.ngrams()
        output: freqdict is a dictionary containing n-grams and corresponding frequency
    """
    freqdict = {}
    for ngram in ngrams:
        if ngram in freqdict.keys():
            freqdict[ngram] += 1
        else:
            freqdict.update({ngram: 1})
    return freqdict


def sort_freqdist(fdist,n):
    """
        Function sort_freqdist() is for sorting the frequency of every ngram and get the top n ngrams based on the number of occurrences.
        input:  fdist is a dictionary containing n-grams and corresponding frequency
        output: list of top n ngrams.
    """
    sorted_list = sorted(fdist.items(), key=lambda item: item[1], reverse=True)
    return sorted_list[:n]

### Read files
Lowercase all the text in signal-news when opening the first file.  
Save the positive and negative words as dictionaries for higher efficiency.

In [4]:
# Read file
t1 = time.time()
with open("signal-news1/signal-news1.jsonl", 'r') as f:
    content = []
    for line in f:
        signal = json.loads(line)
        content.append(signal['content'].lower())   # Lowercase all the text in signal-news when opening the first file.

with open("signal-news1/opinion-lexicon-English/negative-words.txt") as f1, open("signal-news1/opinion-lexicon-English/positive-words.txt") as f2:
    neg_word = f1.read().strip().split("\n")
    pos_word = f2.read().strip().split("\n")

"""Save the positive and negative words as dictionaries for higher efficiency."""
opinion_dict = {}
for word in neg_word:
    opinion_dict.update({word: -1})
for word in pos_word:
    opinion_dict.update({word: 1})
# print(opinion_dict)


### Part A: Text preprocessing  
Use the function difined in the beginning remove URLs first, then non-alphanumeric characters, then 1 character and digits.  
Use **WordNetLemmatizer().lemmatize()** to lemmatize word one by one.

In [5]:
# Part A - 1
newcontent = []
for eachline in content:
    eachline = remove_url(eachline)
    eachline = remove_nonalpha(eachline)
    eachline = remove_character(eachline)
    eachline = remove_digit(eachline)
    newcontent.append(eachline)
# print(newcontent[0])

# Part A - 2
wnl = nltk.WordNetLemmatizer()
lemma_content = []
for line in newcontent:
    line = line.split(' ')
    lemma_word = []
    for word in line:
        lemma_word.append(wnl.lemmatize(word))
    lemma_content.append(lemma_word)
# print(lemma_content[0])

### Part B: N-grams
Compute the number of tokens and vocabulary size  
Use **sort_freqdist()** to get the list of top 25 trigrams based on the number of occurrences on the entire corpus.  
Use **count_opinion_word()** to compute the number of positive and negative word based on the lists of positive and negative words provided.  
Use **count_opinion_word()** to get the number of positive and negative wotd in each news story and then compare them to get the result.

In [6]:
# Part B - 1
print("======Part B======")
all_word = []
for line in lemma_content:
    for word in line:
        all_word.append(word)

N = len(all_word)
V = len(set(all_word))
print('The number of tokens is: ', N)
print('The vocabulary size is: ', V)
print()

# Part B - 2
trigram = nltk.ngrams(all_word, 3)
fdist = compute_freq_dist(trigram)
top_25 = sort_freqdist(fdist, 25)

print("The top 25 trigrams are:")
for f in top_25:
    print(f)
print()

# Part B - 3
num_neg, num_pos = count_opinion_word(all_word, opinion_dict)
print("The number of positive word counts is: ", num_pos)
print("The number of negative word counts is: ", num_neg)
print()

# Part B - 4
pos_news = 0
neg_news = 0
for eachline in lemma_content:
    num_neg, num_pos = count_opinion_word(eachline, opinion_dict)
    if num_pos > num_neg:
        pos_news += 1
    elif num_pos < num_neg:
        neg_news += 1
print("The number of news stories with more positive words is: ", pos_news)
print("The number of news stories with more negative words is: ", neg_news)
print()

======Part B======
The number of tokens is  5641693
The vocabulary size is  90177

The top 25 trigrams are:
(('one', 'of', 'the'), 2438)
(('on', 'share', 'of'), 2093)
(('on', 'the', 'stock'), 1567)
(('a', 'well', 'a'), 1427)
(('in', 'research', 'report'), 1417)
(('samsung', 'samsung', 'samsung'), 1400)
(('in', 'research', 'note'), 1375)
(('the', 'united', 'state'), 1226)
(('for', 'the', 'quarter'), 1222)
(('average', 'price', 'of'), 1193)
(('research', 'report', 'on'), 1177)
(('research', 'note', 'on'), 1138)
(('share', 'of', 'the'), 1133)
(('the', 'end', 'of'), 1130)
(('in', 'report', 'on'), 1124)
(('earnings', 'per', 'share'), 1121)
(('cell', 'phone', 'plan'), 1073)
(('phone', 'plan', 'detail'), 1070)
(('according', 'to', 'the'), 1066)
(('of', 'the', 'company'), 1064)
(('buy', 'rating', 'to'), 1016)
(('appeared', 'first', 'on'), 995)
(('moving', 'average', 'price'), 995)
(('day', 'moving', 'average'), 993)
(('nokia', 'nokia', 'nokia'), 984)

The number of positive word counts is:  17

### Part C: Language models 
Use **nltk.ngrams(), compute_freq_dist()** to compute language models for trigrams based on the first 16,000 rows.  
Use **sort_freqdist()** to get the dictionary in descending order of the frequency and produce a sentence of 10 words beginning with the bigram "is this".  
Use Laplace Smoothing to deal with new words and log transformation to get the perplexity by evaluating on the remaining rows of the corpus(rows 16,001+).

In [7]:
# Part C - 1
print("======Part C======")
word_train = []
for line in lemma_content[:16000]:
    for word in line:
        word_train.append(word)
trigram_train = nltk.ngrams(word_train, 3)
trigram_dist = compute_freq_dist(trigram_train)
trigram_dict = sort_freqdist(trigram_dist, len(trigram_dist))
bigram_train = nltk.ngrams(word_train, 2)
bigram_dist = compute_freq_dist(bigram_train)


# print(trigram_dist[0])
sentence = ['is', 'this']
i = 0
"""
    Find the most common trigram begin with the given bigram, 
    append the third word into the sentence. 
"""
while i < 8:
    for j in range(len(trigram_dict)):
        if sentence[i] == trigram_dict[j][0][0] and sentence[i+1] == trigram_dict[j][0][1]:
            sentence.append(trigram_dict[j][0][2])
            i += 1
            break
print('the sentence beginning with "is this":\n', ' '.join(sentence))
print()

# Part C - 2
tri_test = []
for line in lemma_content[16000:]:
    word_test = []
    for word in line:
        word_test.append(word)
    tri_test.extend(list(nltk.ngrams(word_test, 3)))
# print(tri_test[0])

"""
    Compute the perplexity of remaining rows based on the language models from the first 16000 rows.
    Use Laplace Smoothing to deal with new words.
    Use log transformation to make the perplexity readable.
"""
n = len(tri_test)
pp_1 = 0
for tri in tri_test:
    c1 = trigram_dist.get(tri, 0)
    c2 = bigram_dist.get((tri[0], tri[1]), 0)
    pp_1 += math.log((c2 + n) / (c1 + 1))
pp = pp_1 / n
print("The perplexity of the remaining rows is: ", pp)
print()

print("This program takes ", time.time() - t1, "seconds")

======Part C======
the sentence beginning with "is this":
 is this the company ha market capitalization of billion and

The perplexity of the remaining rows is:  12.749818353544384

This program takes  143.17313289642334 seconds
